# CLUSTERING 



In [27]:
import pandas as pd

df = pd.read_csv('data/AirQuality.csv', delimiter=';')

df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Time'] = pd.to_datetime(df['Time'], format='%H.%M.%S').dt.time

# Añadir columna con el día de la semana
df['DiaSemana'] = df['Date'].dt.day_name()

# Convertir la hora en partes del día
def time_to_period(hour):
    if 6 <= hour < 12:
        return 'Dia'
    elif 12 <= hour < 18:
        return 'Tarde'
    else:
        return 'Noche'

# Aplicar la función para categorizar las horas
df['PeriodoDia'] = df['Time'].apply(lambda t: time_to_period(t.hour))

# Eliminar columnas innecesarias o problemáticas
df = df.drop(columns=['Unnamed: 15', 'Unnamed: 16', 'Time'], errors='ignore')

# Convertir todas las columnas numéricas a tipo numérico
df = df.apply(pd.to_numeric, errors='ignore')  # Convierte columnas a numéricas donde sea posible

# Redondear todos los valores numéricos a enteros
df = df.round(0).astype('Int64', errors='ignore')  # Cambia a 'Int64' para trabajar con NaNs y enteros

# Establecer formato de visualización sin decimales
pd.set_option('display.float_format', '{:.0f}'.format)

df.reset_index(drop=True, inplace=True)

# Eliminar filas con valores faltantes
df_cleaned = df.dropna()  # Elimina filas con NaN
df_cleaned = df.fillna(0)  # Rellena NaN con 0

df_cleaned.info()
df_cleaned.head(20)  # Mostrar las primeras filas

# Mostrar el resultado
print(df)
print(df.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           9471 non-null   Int64 
 1   CO(GT)         9471 non-null   object
 2   PT08.S1(CO)    9471 non-null   Int64 
 3   NMHC(GT)       9471 non-null   Int64 
 4   C6H6(GT)       9471 non-null   object
 5   PT08.S2(NMHC)  9471 non-null   Int64 
 6   NOx(GT)        9471 non-null   Int64 
 7   PT08.S3(NOx)   9471 non-null   Int64 
 8   NO2(GT)        9471 non-null   Int64 
 9   PT08.S4(NO2)   9471 non-null   Int64 
 10  PT08.S5(O3)    9471 non-null   Int64 
 11  T              9471 non-null   object
 12  RH             9471 non-null   object
 13  AH             9471 non-null   object
 14  DiaSemana      9471 non-null   object
 15  PeriodoDia     9471 non-null   object
dtypes: Int64(9), object(7)
memory usage: 1.2+ MB
                      Date CO(GT)  PT08.S1(CO)  NMHC(GT) C6H6(GT)  \


C:\Users\Ikasle\AppData\Local\Temp\ipykernel_11540\214510921.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')  # Convierte columnas a numéricas donde sea posible


In [25]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


#Seleccionar las columnas relevantes de gases
gases_contaminantes = ['CO(GT)', 'NMHC(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)']

# Convertir las columnas de gases contaminantes a tipo numérico, manejando errores
df[gases_contaminantes] = df[gases_contaminantes].apply(pd.to_numeric, errors='coerce')

# Calcular la media de los gases contaminantes para cada fila
df['index'] = df[gases_contaminantes].mean(axis=1)